# Facial Emotion Recognition using Deep Learning

The goal of the project is to develop a deep learning model that can accurately classify facial expressions into one of six categories: Angry, Disgust, Fear, Happy, Sad, and Surprise. To fulfill the scope of the project we separated the neutral expressions. We will use convolutional neural networks (CNNs) to train our model, as they have been shown to be effective in image classification tasks. Our approach involves training a CNN model from scratch using the FER-2013 dataset.

## Data
The FER-2013 dataset consists of 48x48 pixel grayscale images of faces that have been centered and that occupy a similar amount of space in each image. 

## Importing libraries

Import dataset, assign label, and convert img to array.

In [20]:
import os
import cv2
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from keras.callbacks import Callback

# path to folders
dataset_path = 'dataset'

# emotions and respective folders
emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise']

def load_img_and_lbl(dataset_type):
    faces = []
    labels = []
    # iterating through file
    for emotion_index, emotion in enumerate(emotions):
        emotion_path = os.path.join(dataset_path, dataset_type, emotion)
        for img_name in os.listdir(emotion_path):
            img_path = os.path.join(emotion_path, img_name)
            
            # load image and convert to grayscale (designed redundantly)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (64,64))
            
            faces.append(img)
            labels.append(emotion_index)
            
    # convert lists to numpyArrays
    faces = np.asarray(faces)
    labels = np.asarray(labels)

    # normalizing images
    faces = faces / 255.0

    # convert labels
    labels = to_categorical(labels, num_classes=len(emotions))
    
    # reshape for modell
    faces = faces.reshape(faces.shape[0], 64, 64, 1)
    
    return faces, labels

# load training- and testdata
x_train, y_train = load_img_and_lbl('train')
x_test, y_test = load_img_and_lbl('test')

print("Form of training data:", x_train.shape, y_train.shape)

Form of training data: (23744, 64, 64, 1) (23744, 6)


In [26]:
from tqdm import tqdm

class TQDMcallBack(Callback):
    def on_train_begin(self, logs={}):
        self.epochs = self.params['epochs']
        self.tqdm_bar = None
        
    def on_epoch_begin(self, epoch, logs={}):
        self.tqdm_bar = tqdm(total=self.params['steps'], desc=f'Epoch {epoch + 1}/{self.epochs}')

    def on_batch_end(self, batch, logs={}):
        if self.tqdm_bar is not None:
            self.tqdm_bar.update(1)
            self.tqdm_bar.set_postfix_str(f"loss: {logs.get('loss', '?')}, acc: {logs.get('accuracy', '?')}")

    def on_epoch_end(self, epoch, logs={}):
        if self.tqdm_bar is not None:
            self.tqdm_bar.close()


def model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),

        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),

        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPool2D(pool_size=(2, 2)),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

#create model
input_shape = (64, 64, 1)
num_classes = len(emotions)
mod = model(input_shape, num_classes)

## Train Model

In [27]:
history = mod.fit(
    x_train, y_train,
    batch_size = 64,
    epochs = 50,
    validation_data=(x_test, y_test),
    callbacks=[TQDMcallBack()]
    )

2025-07-17 12:09:43.743596: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 389021696 exceeds 10% of free system memory.


Epoch 1/50


  1/371 ━━━━━━━━━━━━━━━━━━━━ 10:58 2s/step - accuracy: 0.0938 - loss: 1.8035

  2/371 ━━━━━━━━━━━━━━━━━━━━ 58s 158ms/step - accuracy: 0.1562 - loss: 1.7752

  3/371 ━━━━━━━━━━━━━━━━━━━━ 58s 160ms/step - accuracy: 0.1840 - loss: 1.7544

...

226/371 ━━━━━━━━━━━━━━━━━━━━ 30s 211ms/step - accuracy: 0.6174 - loss: 0.9762

227/371 ━━━━━━━━━━━━━━━━━━━━ 30s 210ms/step - accuracy: 0.6174 - loss: 0.9763

228/371 ━━━━━━━━━━━━━━━━━━━━ 30s 210ms/step - accuracy: 0.6174 - loss: 0.9764

229/371 ━━━━━━━━━━━━━━━━━━━━ 29s 210ms/step - accuracy: 0.6174 - loss: 0.9764

KeyboardInterrupt: 